# 01-11: AbortController - 取消异步操作AbortController 是一个用于取消异步操作的现代 Web API。

In [ ]:
// ========== 1. AbortController 基础 ==========
const controller = new AbortController();
const signal = controller.signal;
// 监听取消事件
signal.addEventListener('abort', () => {
  console.log('Aborted!');
});
// 触发取消
controller.abort();
console.log('Signal aborted:', signal.aborted); // true

In [ ]:
// ========== 2. 取消 Fetch 请求 ==========
async function fetchWithTimeout(url: string, timeout: number = 5000) {
  const controller = new AbortController();
  const timeoutId = setTimeout(() => controller.abort(), timeout);
  
  try {
    const response = await fetch(url, { signal: controller.signal });
    clearTimeout(timeoutId);
    return response;
  } catch (error) {
    clearTimeout(timeoutId);
    if (error instanceof Error && error.name === 'AbortError') {
      throw new Error('Request timeout');
    }
    throw error;
  }
}
// 使用示例
// fetchWithTimeout('https://api.example.com/data', 3000);

In [ ]:
// ========== 3. 取消多个请求 ==========
class RequestManager {
  private controller: AbortController | null = null;
  
  async fetchAll(urls: string[]) {
    // 取消之前的请求
    this.cancelAll();
    
    this.controller = new AbortController();
    const { signal } = this.controller;
    
    try {
      const promises = urls.map(url => 
        fetch(url, { signal }).then(r => r.json())
      );
      return await Promise.all(promises);
    } catch (error) {
      if (error instanceof Error && error.name === 'AbortError') {
        console.log('All requests cancelled');
        return [];
      }
      throw error;
    }
  }
  
  cancelAll() {
    if (this.controller) {
      this.controller.abort();
      this.controller = null;
    }
  }
}
const manager = new RequestManager();
// manager.fetchAll(['https://api1.com', 'https://api2.com']);

In [ ]:
// ========== 4. 在 Node.js 中使用 ==========
// 从 Node.js v15.4.0 开始支持
import { setTimeout } from 'timers/promises';
async function cancellableOperation(signal: AbortSignal) {
  return new Promise<void>((resolve, reject) => {
    const timeout = setTimeout(() => {
      console.log('Operation completed');
      resolve();
    }, 5000);
    
    signal.addEventListener('abort', () => {
      clearTimeout(timeout);
      reject(new Error('Operation aborted'));
    });
  });
}
// 使用示例
// const controller = new AbortController();
// cancellableOperation(controller.signal);
// setTimeout(() => controller.abort(), 2000);

In [ ]:
// ========== 5. 链式取消信号 ==========
// 使用 AbortSignal.any (Node.js v20.3.0+, Chrome v116+)
function createLinkedSignal(...signals: AbortSignal[]): AbortSignal {
  const controller = new AbortController();
  
  for (const signal of signals) {
    if (signal.aborted) {
      controller.abort(signal.reason);
      break;
    }
    signal.addEventListener('abort', () => controller.abort(signal.reason), {
      once: true
    });
  }
  
  return controller.signal;
}
// 使用示例
// const timeoutController = new AbortController();
// const userController = new AbortController();
// setTimeout(() => timeoutController.abort(), 5000);
// const linked = createLinkedSignal(timeoutController.signal, userController.signal);